In [1]:
import numpy as np 
import pandas as pd
import sys
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

In [2]:
ACC=pd.read_csv('1571132161_A013E1\ACC.csv')
BVP=pd.read_csv('1571132161_A013E1\BVP.csv') ##Blood Volume Pulse
EDA=pd.read_csv('1571132161_A013E1\EDA.csv')
HR=pd.read_csv('1571132161_A013E1\HR.csv')
IBI=pd.read_csv('1571132161_A013E1\IBI.csv') #inter-beat-interval

In [3]:
a=BVP.loc[::64]
b=ACC.loc[::32]
c=EDA.loc[::4]
a=a.reset_index()
b=b.reset_index()
c=c.reset_index()
DataSet=pd.DataFrame(columns=['BVP','ACC','EDA','HR','EM1','EM2'])
DataSet['BVP'] = a.iloc[:,1]
DataSet['ACC'] = b.iloc[:,1]
DataSet['EDA'] = c.iloc[:,1]
DataSet['HR']=HR
DataSet=DataSet.drop(DataSet.index[len(HR):len(DataSet)])

In [4]:
DataSet['EM1'].iloc[0:int(len(DataSet)/3)]= 0
DataSet['EM2'].iloc[0:int(len(DataSet)/3)]= -1
DataSet['EM1'].iloc[int(len(DataSet)/3):2*int(len(DataSet)/3)]= 2
DataSet['EM2'].iloc[int(len(DataSet)/3):2*int(len(DataSet)/3)]= -2
DataSet['EM1'].iloc[2*int(len(DataSet)/3):int(len(DataSet))]= 4
DataSet['EM2'].iloc[2*int(len(DataSet)/3):int(len(DataSet))]= -2

C:\Users\PCI\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
DataSet.to_csv('Dataset.csv')

In [19]:
DataSet.head()

,BVP,ACC,EDA,HR,EM1,EM2
0,64.00,32.0,4.000000,1.00,0,-1
1,85.71,38.0,0.039712,86.00,0,-1
2,148.81,44.0,0.008967,97.00,0,-1
3,11.23,23.0,0.008967,93.00,0,-1
4,47.26,17.0,0.017934,84.25,0,-1


In [5]:
from keras.models import Sequential # 케라스의 Sequential()을 임포트
from keras.layers import Dense # 케라스의 Dense()를 임포트
from keras import optimizers # 케라스의 옵티마이저를 임포트

Using TensorFlow backend.


In [24]:
df=pd.DataFrame(columns=['BVP','ACC','EDA','HR'])
df['BVP']=DataSet['BVP']
df['ACC']=DataSet['ACC']
df['EDA']=DataSet['EDA']
df['HR']=DataSet['HR']

df_em1=pd.DataFrame(columns=['EM1'])
df_em2=pd.DataFrame(columns=['EM2'])
df_em1['EM1']=DataSet['EM1']
df_em2['EM2']=DataSet['EM2']

In [38]:
X=df

In [39]:
y=df_em1

In [46]:
from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values
Y

C:\Users\PCI\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=uint8)

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=1) 
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 4), (179, 4), (712, 3), (179, 3))

In [48]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

model = Sequential()
sgd=optimizers.SGD(lr=0.01)
model.add(Dense(64,input_shape=(4,),activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                320       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 195       
Total params: 4,675
Trainable params: 4,675
Non-trainable params: 0
_________________________________________________________________


In [56]:
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)

Train on 712 samples, validate on 179 samples
Epoch 1/100
712/712 [==============================] - 0s 104us/step - loss: 0.4472 - acc: 0.8118 - val_loss: 0.7726 - val_acc: 0.7207
Epoch 2/100
712/712 [==============================] - 0s 79us/step - loss: 0.4693 - acc: 0.8076 - val_loss: 0.7401 - val_acc: 0.7486
Epoch 3/100
712/712 [==============================] - 0s 106us/step - loss: 0.4390 - acc: 0.8272 - val_loss: 0.7999 - val_acc: 0.7486
Epoch 4/100
712/712 [==============================] - 0s 98us/step - loss: 0.4578 - acc: 0.8146 - val_loss: 0.7568 - val_acc: 0.6927
Epoch 5/100
712/712 [==============================] - 0s 93us/step - loss: 0.4414 - acc: 0.8315 - val_loss: 0.8122 - val_acc: 0.7318
Epoch 6/100
712/712 [==============================] - 0s 79us/step - loss: 0.4283 - acc: 0.8329 - val_loss: 0.7252 - val_acc: 0.7430
Epoch 7/100
712/712 [==============================] - 0s 71us/step - loss: 0.4080 - acc: 0.8441 - val_loss: 0.7570 - val_acc: 0.7654
Epoch 8/100
71

712/712 [==============================] - 0s 74us/step - loss: 0.3369 - acc: 0.8624 - val_loss: 0.6443 - val_acc: 0.7877
Epoch 62/100
712/712 [==============================] - 0s 65us/step - loss: 0.3696 - acc: 0.8638 - val_loss: 0.6332 - val_acc: 0.7933
Epoch 63/100
712/712 [==============================] - 0s 65us/step - loss: 0.3404 - acc: 0.8624 - val_loss: 0.6135 - val_acc: 0.7765
Epoch 64/100
712/712 [==============================] - 0s 62us/step - loss: 0.3582 - acc: 0.8413 - val_loss: 0.6564 - val_acc: 0.7933
Epoch 65/100
712/712 [==============================] - 0s 68us/step - loss: 0.3323 - acc: 0.8610 - val_loss: 0.6753 - val_acc: 0.7877
Epoch 66/100
712/712 [==============================] - 0s 69us/step - loss: 0.3364 - acc: 0.8596 - val_loss: 0.6830 - val_acc: 0.7709
Epoch 67/100
712/712 [==============================] - 0s 60us/step - loss: 0.3472 - acc: 0.8638 - val_loss: 0.6473 - val_acc: 0.7598
Epoch 68/100
712/712 [==============================] - 0s 67us/step

In [57]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy = {:.2f}".format(accuracy))

179/179 [==============================] - 0s 50us/step
Accuracy = 0.72


In [51]:
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix

y_pred = model.predict(X_test)
y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

              precision    recall  f1-score   support

           0       0.69      0.63      0.65        59
           1       0.60      0.53      0.57        49
           2       0.72      0.83      0.77        71

   micro avg       0.68      0.68      0.68       179
   macro avg       0.67      0.66      0.66       179
weighted avg       0.68      0.68      0.68       179

[[37 11 11]
 [11 26 12]
 [ 6  6 59]]
